In [ ]:
! pip install google
! pip install transformers

In [11]:
import bs4
import requests
from bs4 import BeautifulSoup 
import numpy as np
import googlesearch
import pandas as pd
import transformers
import spacy
import re

In [12]:
def docs_to_sentences(df_docs, split=r"\n"):
    number_docs = df_docs.shape[0]
    df_sentences = pd.DataFrame(columns=['doc_id','sentence'])
    for i in range(number_docs):
        text = df_docs.text[i]
        replace_dict = {'?«' :  '«', '(' :  '', ')' : '', ':' : ',', '.' : ',', ',,,' : ',', '"':''}
        for x,y in replace_dict.items():
            text = text.replace(x, y)
        text = text.lower()   
        sentences = re.split(split, text)
        len_sentences = len(sentences)   
        doc_id = [i] * (len_sentences)
        doc_sentences = pd.DataFrame({'doc_id' : doc_id, 'sentence' : sentences})
        df_sentences = df_sentences.append(doc_sentences)   
    df_sentences = df_sentences[df_sentences.sentence != '']
    df_sentences.reset_index(drop=True, inplace=True)  
    return df_sentences

In [13]:
def poem_generator(sentences, word, n_sents=5):
        nlp = spacy.load("en_core_web_sm")
        init_str = nlp(word)
        sup_index= sentences.shape[0]
        poem_id = int()
        poem =[]
        for i in range(n_sents):
            rand_sent_index = np.random.randint(0, sup_index, size=30)
            sent_list = list(sentences.sentence.iloc[rand_sent_index])
            docs = nlp.pipe(sent_list)
            sim_list = []
            for sent in docs:
                similarity = (init_str.similarity(sent))
                sim_list.append(similarity)
            df_1 = pd.DataFrame({'similarity' : sim_list, 'doc_id' : sentences.doc_id.iloc[rand_sent_index] }, index=rand_sent_index)   
            df_1 = df_1[df_1.doc_id != poem_id]
            df_1.sort_values(by='similarity', inplace=True, ascending=False)
            sent_index= df_1.index[0]
            sent = sentences.sentence[sent_index]
            replace_dict = {'\n' :  '', '\r' :  ''}
            for x,y in replace_dict.items():
                sent = sent.replace(x, y)
            poem.append(sent)    
            poem_id = df_1.doc_id.iloc[0]
            init_str = nlp(sent)  
        str_poem = ("\n".join(poem)) 
        return str_poem

In [14]:
def search(query):
  googleSearch = googlesearch.search(query, num=20, lang='en')
  Bart_Model = transformers.pipeline('summarization', model='facebook/bart-large-xsum')
  notEnough = True
  all_texts = []
  while notEnough:
    for link in googleSearch:
      if not notEnough:
          break
      reqs = requests.get(link)
      soup = BeautifulSoup(reqs.text, 'html.parser')
      for p in soup.find_all('p'):
        if not notEnough:
          break
        for childp in p:
          cont = str(childp.string)
          if len(cont.split(' ')) > 40:
            cont = Bart_Model(cont, min_length=20, max_length=40, do_sample=False)
            cont = cont[0]['summary_text']
          elif 20 > len(cont.split(' ')):
            continue
          if cont in all_texts:
            continue
          all_texts.append(cont)
          cony = ' '
          if len(all_texts) >= 70:
            notEnough = False
            break

  all_texts_df = pd.DataFrame(all_texts,columns=['text'])
  sentences = docs_to_sentences(all_texts_df, split=r"\n")
  poem = poem_generator(sentences,word=query)
  return poem


In [18]:


res = search("Russia")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  del sys.path[0]


In [19]:
print(res)

nato on thursday condemned plans to hold referendums in russian-occupied parts of ukraine, calling them a “sham” that the alliance will not recognize,  
clooney, who is advising kyiv on finding accountability for ukrainian victims, called for world leaders to help ukrainians rebuild by supporting a draft un resolution to establish a framework for a compensation commission, 
kuleba urged un member states to help his country fight russian aggression and win justice by holding russia accountable for its invasion of ukraine, 
a bipartisan group of lawmakers asked the pentagon to send advanced drones to ukraine in a letter sent to secretary of defense lloyd austin on sept, 21 and obtained by cnn,
us deputy secretary of state antony blinken has warned that russia is ready to use all of its weapons against ukraine,


In [17]:
# def poem_generator(sentences, word, n_sents=4):
#         nlp = spacy.load("en_core_web_sm")
#         init_str = nlp(word)
#         sup_index= sentences.shape[0]
#         poem_id = int()
#         poem =[]
#         for i in range(n_sents):
#             rand_sent_index = np.random.randint(0, sup_index, size=30)
#             sent_list = list(sentences.sentence.iloc[rand_sent_index])
#             docs = nlp.pipe(sent_list)
#             sim_list = []
#             for sent in docs:
#                 similarity = (init_str.similarity(sent))
#                 sim_list.append(similarity)
#             df_1 = pd.DataFrame({'similarity' : sim_list, 'doc_id' : sentences.doc_id.iloc[rand_sent_index] }, index=rand_sent_index)   
#             df_1 = df_1[df_1.doc_id != poem_id]
#             df_1.sort_values(by='similarity', inplace=True, ascending=False)
#             sent_index= df_1.index[0]
#             sent = sentences.sentence[sent_index]
#             replace_dict = {'\n' :  '', '\r' :  ''}
#             for x,y in replace_dict.items():
#                 sent = sent.replace(x, y)
#             poem.append(sent)    
#             poem_id = df_1.doc_id.iloc[0]
#             init_str = nlp(sent)  
#         str_poem = ("\n".join(poem)) 
#         return str_poem

In [9]:
# def search(query):
#   googlenews = GoogleNews()

#   googlenews.search(query)
#   googlenews.get_page(1)
#   links = googlenews.get_links()[:5]
#   sizes = []
#   all_texts = []
#   for link in links:
#     reqs = requests.get(link)
#     soup = BeautifulSoup(reqs.text, 'html.parser')
#     txt = []
#     for p in soup.find_all('p'):
#       for childp in p:
#         cont = str(childp.string)
#         if len(cont) > 20:
#           txt.append(cont)
#     txt = " \n ".join(txt)
#     sizes.append(len(txt))
#     all_texts.append(txt)

#   all_texts = np.array(all_texts)

#   return all_texts, sizes, links